In [1]:
# In your Jupyter notebook (TEST_Daily_Analysis.ipynb)
import sys
from pathlib import Path

# Add the project root directory to Python path
ROOT_DIR = Path.cwd().parent  # Goes up one level from 'scanner' to 'stockbot4'
if str(ROOT_DIR) not in sys.path:
    sys.path.append(str(ROOT_DIR))

In [2]:
import pandas as pd
from data.random_data import RandomOHLCV
# from data.ohlcv import ServeNewOHLCV
import strategies.ta as ta
import strategies.signals as sig
from frame import Frame

periods = 365

ohlcv = RandomOHLCV( 
    freq      = '15 min', 
    head_max  = 0.3, 
    tail_max  = 0.3, 
    start     = '2024',           
    open_val  = 100.00,           
    periods   = periods, 
    open_rng  = (-0.4, 0.4), 
    close_rng = (-0.4, 0.4), 
    vol_rng   = (-1, 1),
    volatility_rng  = (0, 0.02),
    volatility_dur  = 3,
    volatility_freq = 50
).get_dataframe()


f = Frame('TSLA', run_ta_on_load=True)
f.load_ohlcv(ohlcv)
f.add_ta(ta.ATR(span=50), {'dash': 'solid', 'color': 'cyan', 'width': 1}, row=3, chart_type='')
# f.add_ta(ta.MA('close', 9),  {'dash': 'solid', 'color': 'magenta', 'width': 2}) # appends ta to the ta list
# f.add_ta(ta.MA('close', 21), {'dash': 'solid', 'color': 'cyan', 'width': 1})
f.add_ta(ta.MA('close', 50), {'dash': 'solid', 'color': 'magenta', 'width': 2}, row=1)
f.add_ta(ta.MA('close', 100), {'dash': 'solid', 'color': 'cyan', 'width': 3}, row=1)
f.add_ta(ta.MA('close', 150), {'dash': 'solid', 'color': 'yellow', 'width': 5}, row=1)
f.add_ta(ta.MA('volume', 10), {'dash': 'solid', 'color': 'yellow', 'width': 1}, row=2)
# f.add_ta(ta.MACD(fast=12, slow=26, signal=9) , 
#          [{'dash': 'solid', 'color': 'purple', 'width': 2}, # MACD
#           {'dash': 'solid', 'color': 'pink', 'width': 2},   # signal
#           {'color': 'black'}], 'macd', 2) # histogram

f.add_ta(ta.HPLP(hi_col='high', lo_col='low', span=10), 
        [{'color': 'green', 'size': 10}, # high points
          {'color': 'red', 'size': 10}], # low points
          chart_type = 'points')

f.add_ta(ta.MicroTrendline(name='MTREND', pointsCol='HP_hi_10', atrCol='ATR_50', slopeDir='down', slopeToleranceATR=2, projectionPeriod=10),
            {'dash': 'solid', 'color': 'rgb(173, 90, 255)', 'width': 1}, 
            chart_type = 'trendlines')

# f.add_ta(ta.SupRes(hi_point_col='HP_hi_10', lo_point_col='LP_lo_10', atr_col='ATR_50', tolerance=1),
#             [{'dash': 'solid', 'color': 'green', 'fillcolour': "rgba(0, 255, 0, 0.1)", 'width': 2}, # support # green = rgba(0, 255, 0, 0.1)
#             {'dash': 'solid', 'color': 'red', 'fillcolour': "rgba(255, 0, 0, 0.1)", 'width': 2}], # resistance # red = rgba(255, 0, 0, 0.1)
#             chart_type = 'support_resistance') 

f.add_ta(ta.ConsolidationZone(hp_column='HP_hi_10', lp_column='LP_lo_10', atr_column='ATR_50', price_tolerance=0.001, max_points_between=1, height_width_ratio=50, limit_zones=2),
            [{'color': "rgba(225, 182, 30, 0.5)", 'fillcolour': "rgba(225, 182, 30, 0.1)", 'width': 2}, # support # green = rgba(0, 255, 0, 0.1)
            {'color': "rgba(225, 182, 30, 0.1)", 'fillcolour': "rgba(225, 182, 30, 0.1)", 'width': 2}], # resistance # red = rgba(255, 0, 0, 0.1)
            chart_type = 'cons')

# f.setup_chart() 

f.backtest_setup(
    start=-50,  # This will slice the data from 0:400
    end=-1,    # We'll test up to row 450
    save_snapshots=True
)

f.backtest_run(clear_snapshots=True)

Tested rows 49/49 - Date: 2024-01-04 18:45:00 (location 363)


In [3]:
f.backtest_data.columns

Index(['open', 'high', 'low', 'close', 'volume', 'ATR_50', 'MA_cl_50',
       'MA_cl_100', 'MA_cl_150', 'MA_vo_10', 'HP_hi_10', 'LP_lo_10',
       'MTREND_DOWN_1', 'MTREND_DOWN_2', 'MTREND_DOWN_3', 'MTREND_DOWN_4',
       'MTREND_DOWN_5', 'CONS_UPPER_1', 'CONS_LOWER_1', 'MTREND_DOWN_6',
       'CONS_UPPER_2', 'CONS_LOWER_2', 'MTREND_DOWN_7'],
      dtype='object')

In [4]:
f.run_snapshots(0,50, plot=True, display_df=False, sleep_time=0.1, width=2000, height=1500)
# f.get_snapshot(22, plot=True, width=2000, height=1500)

Viewing snapshot 50/50 - Date: 2024-01-04 18:45:00 (location 363)


In [5]:
f.backtest_data

,open,high,low,close,volume,ATR_50,MA_cl_50,MA_cl_100,MA_cl_150,MA_vo_10,...,MTREND_DOWN_2,MTREND_DOWN_3,MTREND_DOWN_4,MTREND_DOWN_5,CONS_UPPER_1,CONS_LOWER_1,MTREND_DOWN_6,CONS_UPPER_2,CONS_LOWER_2,MTREND_DOWN_7
date,,,,,,,,,,,,,,,,,,,,,
2024-01-01 00:00:00,100.14,100.37,99.87,99.98,100,0.500000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,100.49,98.77,NaN,100.49,98.77,NaN
2024-01-01 00:15:00,100.31,100.48,99.87,99.94,100,0.555000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,100.49,98.77,NaN,100.49,98.77,NaN
2024-01-01 00:30:00,99.80,99.84,99.43,99.61,42,0.506667,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,100.49,98.77,NaN,100.49,98.77,NaN
2024-01-01 00:45:00,99.65,100.16,99.43,100.01,58,0.562500,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,100.49,98.77,NaN,100.49,98.77,NaN
2024-01-01 01:00:00,99.90,99.92,99.46,99.51,79,0.542000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,100.49,98.77,NaN,100.49,98.77,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-04 17:45:00,89.03,89.24,88.90,88.93,8,0.448200,89.4898,90.3201,91.833800,7.9,...,NaN,NaN,NaN,91.299259,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-04 18:00:00,88.80,88.96,88.53,88.64,5,0.441000,89.4546,90.2870,91.787067,7.2,...,NaN,NaN,NaN,91.256667,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-04 18:15:00,88.99,89.21,88.73,88.96,5,0.446600,89.4272,90.2565,91.743867,6.4,...,NaN,NaN,NaN,91.214074,NaN,NaN,NaN,NaN,NaN,NaN
